In [1]:
import json
from urllib.parse import urlparse, parse_qs, urljoin

import requests as rq
from bs4 import BeautifulSoup


In [2]:
def get_daylywebtoons():

    webtoon_main_url = WC['TOP_URL']
    res = rq.get(webtoon_main_url)
    main_soup = BeautifulSoup(res.content, 'lxml')
    webtoon_links = [{"title": a_tag.get('title'), "link": urljoin(WC['TOP_URL'], a_tag.get('href'))}
                      for a_tag in main_soup.select('#content a.title')]
    
    return webtoon_links


In [3]:
WC = {'TOP_URL' : 'http://comic.naver.com/webtoon/weekday.nhn'}


In [4]:
webtoons = get_daylywebtoons()
webtoons[:5]


[{'link': 'http://comic.naver.com/webtoon/list.nhn?titleId=183559&weekday=mon',
  'title': '신의 탑'},
 {'link': 'http://comic.naver.com/webtoon/list.nhn?titleId=648419&weekday=mon',
  'title': '뷰티풀 군바리'},
 {'link': 'http://comic.naver.com/webtoon/list.nhn?titleId=602910&weekday=mon',
  'title': '윈드브레이커'},
 {'link': 'http://comic.naver.com/webtoon/list.nhn?titleId=654774&weekday=mon',
  'title': '소녀의 세계'},
 {'link': 'http://comic.naver.com/webtoon/list.nhn?titleId=679519&weekday=mon',
  'title': '대학일기'}]

In [5]:
def get_qs(url, key):
    url_query = urlparse(url).query
    query_dict = parse_qs(url_query)
    value = query_dict[key][0]
    return value


In [6]:
def get_webtoon_id(number):
    webtoon_url = webtoons[number]['link']
    webtoon_title = webtoons[number]['title']
    print('webtoon_title :', webtoon_title)

    webtoon_id = get_qs(webtoon_url, 'titleId')
    print('webtoon_id : ', webtoon_id)
    weekday = get_qs(webtoon_url, 'weekday')
    print('weekday : ', weekday)
    
    return webtoon_id


In [7]:
def get_webtoon_url(number):
    
    webtoon_url = webtoons[number]['link']
    print('webtoon_url :', webtoon_url)
    
    return webtoon_url


In [8]:
def get_start_chapter(n):
    url = get_webtoon_url(n)
    res = rq.get(url)
    startchapter = BeautifulSoup(res.content, 'lxml')
    aa = startchapter.select('#content > table > tr > td.title > a')[0]
    u = aa.get('href')
    url_query = urlparse(u).query
    query_dict = parse_qs(url_query)
    start_chapter = query_dict['no'][0]
    return start_chapter


In [9]:
def get_comment_soup(webtoon_id, chapter, comment_page=1):
    Referer_url='http://m.comic.naver.com/scrolltoon/comment.nhn?titleId='+str(webtoon_id)+'&no='+str(chapter)
    coment_url='http://apis.naver.com/commentBox/cbox/web_naver_list_jsonp.json?ticket=comic&templateId=webtoon&pool=cbox3&_callback=jQuery33104420385964332538_1524620976824&lang=ko&country=KR&objectId='+str(webtoon_id) + '_' + str(chapter)+'&categoryId=&pageSize=15&indexSize=10&groupId=&listType=OBJECT&pageType=more&page='+str(comment_page)+'&initialize=true&userType=&useAltSort=true&replyPageSize=10&sort=NEW&_=1524620976825'
    header = {
            "Host": "apis.naver.com",
            "Referer": Referer_url,
        }

    res = rq.get(coment_url, headers = header)
    soup = BeautifulSoup(res.content, 'lxml')
    return soup


In [10]:
def from_soup_to_maxlpage(tt):
    content_text = tt.select('p')[0].text
    one = content_text.find('(') + 1
    two = content_text.find(');')
    content = json.loads(content_text[one:two])
    max_comment_page = content['result']['pageModel']['totalPages']
    return max_comment_page


In [11]:
def get_comment_ls(soup, chapter, comment_ls):
    content_text = soup.select('p')[0].text
    one = content_text.find('(') + 1
    two = content_text.find(');')
    content = json.loads(content_text[one:two])
    comments = content['result']['commentList']

    for n in range(0,14+1):
        ls = []
        com = comments[n]['contents']
        userid = comments[n]['maskedUserName'] + '('+comments[n]['maskedUserId'] + ')'
        like = comments[n]['sympathyCount']
        dislike = comments[n]['antipathyCount']
        ls = [chapter, userid, com, like, dislike]
        comment_ls.append(ls)
        
    return comment_ls


In [12]:
def get_data(title, webtoon_id, chapter):

    tt = get_comment_soup(webtoon_id, chapter, comment_page=1)
    max_comment_page= from_soup_to_maxlpage(tt)
    comment_ls = []
    
    for comment_page in range(1,max_comment_page+1):      
        soup = get_comment_soup(webtoon_id, chapter, comment_page)
        try:
            get_comment_ls(soup, chapter, comment_ls)
        except :
            pass
        
        
    print('chapter '+str(chapter)+' Load finish')
            
    return comment_ls


In [ ]:
for webtoon_number in range(0, 87+1):
    webtoon_id = get_webtoon_id(webtoon_number)
    start = int(get_start_chapter(webtoon_number))

    empt = [] 
    for ch in range(start-15, start+1) :
        ls=[]
        try:
            ls = get_data(str(webtoon_id), webtoon_id, chapter = ch)         
        except :
                pass
        for i in range(len(ls)):
            empt.append(ls[i])
    df = pd.DataFrame(data = empt ,columns=['chapter', 'user_id', 'comment', 'like', 'dislike'])
    df.to_csv('../data/webtoon_{}.csv'.format(webtoon_number))
    

webtoon_title : 신의 탑
webtoon_id :  183559
weekday :  mon
webtoon_url : http://comic.naver.com/webtoon/list.nhn?titleId=183559&weekday=mon
chapter 365 Load finish
chapter 366 Load finish
chapter 367 Load finish
chapter 368 Load finish
chapter 369 Load finish
chapter 370 Load finish
chapter 371 Load finish
chapter 372 Load finish
chapter 373 Load finish
chapter 374 Load finish
chapter 375 Load finish
chapter 377 Load finish
chapter 378 Load finish
chapter 379 Load finish
chapter 380 Load finish
webtoon_title : 뷰티풀 군바리
webtoon_id :  648419
weekday :  mon
webtoon_url : http://comic.naver.com/webtoon/list.nhn?titleId=648419&weekday=mon
chapter 146 Load finish
chapter 147 Load finish
chapter 148 Load finish
chapter 149 Load finish
chapter 150 Load finish
chapter 151 Load finish
chapter 152 Load finish
chapter 153 Load finish
chapter 154 Load finish
chapter 155 Load finish
chapter 156 Load finish
chapter 157 Load finish
chapter 158 Load finish
chapter 159 Load finish
chapter 160 Load finish
c

chapter -12 Load finish
chapter -11 Load finish
chapter -10 Load finish
chapter -9 Load finish
chapter -8 Load finish
chapter -7 Load finish
chapter -6 Load finish
chapter -5 Load finish
chapter -4 Load finish
chapter -3 Load finish
chapter -2 Load finish
chapter -1 Load finish
chapter 0 Load finish
chapter 2 Load finish
chapter 3 Load finish
webtoon_title : 히어로메이커
webtoon_id :  21815
weekday :  mon
webtoon_url : http://comic.naver.com/webtoon/list.nhn?titleId=21815&weekday=mon
chapter 597 Load finish
chapter 598 Load finish
chapter 599 Load finish
chapter 600 Load finish
chapter 601 Load finish
chapter 602 Load finish
chapter 603 Load finish
chapter 604 Load finish
chapter 605 Load finish
chapter 606 Load finish
chapter 607 Load finish
chapter 608 Load finish
chapter 609 Load finish
chapter 610 Load finish
chapter 611 Load finish
chapter 612 Load finish
webtoon_title : 닥터 하운드
webtoon_id :  700858
weekday :  mon
webtoon_url : http://comic.naver.com/webtoon/list.nhn?titleId=700858&weekd

chapter 30 Load finish
chapter 31 Load finish
chapter 33 Load finish
chapter 34 Load finish
chapter 35 Load finish
chapter 36 Load finish
chapter 37 Load finish
chapter 38 Load finish
chapter 39 Load finish
chapter 40 Load finish
webtoon_title : 덴마
webtoon_id :  119874
weekday :  tue
webtoon_url : http://comic.naver.com/webtoon/list.nhn?titleId=119874&weekday=tue
chapter 1139 Load finish
chapter 1140 Load finish
chapter 1141 Load finish
chapter 1142 Load finish
chapter 1143 Load finish
chapter 1144 Load finish
chapter 1145 Load finish
chapter 1146 Load finish
chapter 1147 Load finish
chapter 1148 Load finish
chapter 1149 Load finish
chapter 1150 Load finish
chapter 1151 Load finish
chapter 1152 Load finish
chapter 1153 Load finish
chapter 1154 Load finish
webtoon_title : 신도림
webtoon_id :  683496
weekday :  tue
webtoon_url : http://comic.naver.com/webtoon/list.nhn?titleId=683496&weekday=tue
chapter 74 Load finish
chapter 75 Load finish
chapter 76 Load finish
chapter 77 Load finish
chapt

chapter 145 Load finish
chapter 146 Load finish
chapter 147 Load finish
chapter 148 Load finish
chapter 149 Load finish
chapter 150 Load finish
chapter 151 Load finish
chapter 152 Load finish
chapter 153 Load finish
chapter 154 Load finish
chapter 155 Load finish
chapter 156 Load finish
chapter 157 Load finish
chapter 158 Load finish
chapter 159 Load finish
webtoon_title : 미스터리 호러 지하철 (재)
webtoon_id :  700361
weekday :  tue
webtoon_url : http://comic.naver.com/webtoon/list.nhn?titleId=700361&weekday=tue
chapter 21 Load finish
chapter 29 Load finish
chapter 31 Load finish
chapter 32 Load finish
chapter 33 Load finish
chapter 36 Load finish
webtoon_title : 패밀리 사이즈
webtoon_id :  626906
weekday :  tue
webtoon_url : http://comic.naver.com/webtoon/list.nhn?titleId=626906&weekday=tue
chapter 364 Load finish
chapter 366 Load finish
chapter 367 Load finish
chapter 369 Load finish
chapter 373 Load finish
chapter 374 Load finish
chapter 375 Load finish
chapter 377 Load finish
chapter 378 Load fin

chapter 119 Load finish
chapter 120 Load finish
webtoon_title : 조선왕조실톡
webtoon_id :  642598
weekday :  wed
webtoon_url : http://comic.naver.com/webtoon/list.nhn?titleId=642598&weekday=wed
chapter 294 Load finish
chapter 295 Load finish
chapter 296 Load finish
chapter 297 Load finish
chapter 298 Load finish
chapter 299 Load finish
chapter 300 Load finish
chapter 301 Load finish
chapter 302 Load finish
chapter 303 Load finish
chapter 304 Load finish
chapter 305 Load finish
chapter 306 Load finish
chapter 309 Load finish
webtoon_title : 미시령
webtoon_id :  697533
weekday :  wed
webtoon_url : http://comic.naver.com/webtoon/list.nhn?titleId=697533&weekday=wed
chapter 32 Load finish
chapter 33 Load finish
chapter 34 Load finish
chapter 35 Load finish
chapter 36 Load finish
chapter 38 Load finish
chapter 39 Load finish
chapter 40 Load finish
chapter 41 Load finish
chapter 42 Load finish
chapter 43 Load finish
chapter 44 Load finish
chapter 45 Load finish
chapter 46 Load finish
webtoon_title : 하

chapter 205 Load finish
chapter 206 Load finish
chapter 207 Load finish
chapter 208 Load finish
chapter 209 Load finish
chapter 210 Load finish
chapter 211 Load finish
chapter 212 Load finish
chapter 214 Load finish
chapter 215 Load finish
chapter 216 Load finish
chapter 217 Load finish
chapter 218 Load finish
chapter 219 Load finish
chapter 220 Load finish
webtoon_title : 기기괴괴
webtoon_id :  557672
weekday :  thu
webtoon_url : http://comic.naver.com/webtoon/list.nhn?titleId=557672&weekday=thu
chapter 215 Load finish
chapter 216 Load finish
chapter 218 Load finish
chapter 219 Load finish
chapter 220 Load finish
chapter 221 Load finish
chapter 223 Load finish
chapter 224 Load finish
chapter 225 Load finish
chapter 226 Load finish
chapter 227 Load finish
chapter 228 Load finish
chapter 229 Load finish
chapter 230 Load finish
webtoon_title : 대학일기
webtoon_id :  679519
weekday :  thu
webtoon_url : http://comic.naver.com/webtoon/list.nhn?titleId=679519&weekday=thu
chapter 186 Load finish


In [13]:
for webtoon_number in range(89, 99+1):
    webtoon_id = get_webtoon_id(webtoon_number)
    start = int(get_start_chapter(webtoon_number))

    empt = [] 
    for ch in range(start-15, start+1) :
        ls=[]
        try:
            ls = get_data(str(webtoon_id), webtoon_id, chapter = ch)         
        except :
                pass
        for i in range(len(ls)):
            empt.append(ls[i])
    df = pd.DataFrame(data = empt ,columns=['chapter', 'user_id', 'comment', 'like', 'dislike'])
    df.to_csv('../data/webtoon_{}.csv'.format(webtoon_number))
    
    

webtoon_title : 타인은 지옥이다
webtoon_id :  708378
weekday :  thu
webtoon_url : http://comic.naver.com/webtoon/list.nhn?titleId=708378&weekday=thu
chapter -1 Load finish
chapter 0 Load finish
chapter 1 Load finish
chapter 2 Load finish
chapter 4 Load finish
chapter 5 Load finish
chapter 6 Load finish
chapter 7 Load finish
chapter 8 Load finish
chapter 9 Load finish
chapter 11 Load finish
chapter 12 Load finish
chapter 13 Load finish
chapter 14 Load finish
webtoon_title : 최강전설 강해효
webtoon_id :  570506
weekday :  thu
webtoon_url : http://comic.naver.com/webtoon/list.nhn?titleId=570506&weekday=thu
chapter 228 Load finish
chapter 229 Load finish
chapter 230 Load finish
chapter 231 Load finish
chapter 232 Load finish
chapter 233 Load finish
chapter 234 Load finish
chapter 235 Load finish
chapter 236 Load finish
chapter 237 Load finish
chapter 238 Load finish
chapter 239 Load finish
chapter 240 Load finish
chapter 241 Load finish
chapter 242 Load finish
chapter 243 Load finish
webtoon_title : 좋아하

In [14]:
for webtoon_number in range(101, 192+1):
    webtoon_id = get_webtoon_id(webtoon_number)
    start = int(get_start_chapter(webtoon_number))

    empt = [] 
    for ch in range(start-15, start+1) :
        ls=[]
        try:
            ls = get_data(str(webtoon_id), webtoon_id, chapter = ch)         
        except :
                pass
        for i in range(len(ls)):
            empt.append(ls[i])
    df = pd.DataFrame(data = empt ,columns=['chapter', 'user_id', 'comment', 'like', 'dislike'])
    df.to_csv('../data/webtoon_{}.csv'.format(webtoon_number))
    
    

webtoon_title : 소유
webtoon_id :  707671
weekday :  thu
webtoon_url : http://comic.naver.com/webtoon/list.nhn?titleId=707671&weekday=thu
chapter -7 Load finish
chapter -6 Load finish
chapter -5 Load finish
chapter -4 Load finish
chapter -3 Load finish
chapter -2 Load finish
chapter -1 Load finish
chapter 0 Load finish
chapter 1 Load finish
chapter 2 Load finish
chapter 3 Load finish
chapter 4 Load finish
chapter 5 Load finish
chapter 6 Load finish
chapter 7 Load finish
chapter 8 Load finish
webtoon_title : 하루 3컷
webtoon_id :  644180
weekday :  thu
webtoon_url : http://comic.naver.com/webtoon/list.nhn?titleId=644180&weekday=thu
chapter 1200 Load finish
chapter 1201 Load finish
chapter 1202 Load finish
chapter 1203 Load finish
chapter 1204 Load finish
chapter 1205 Load finish
chapter 1206 Load finish
chapter 1207 Load finish
chapter 1208 Load finish
chapter 1209 Load finish
chapter 1210 Load finish
chapter 1211 Load finish
chapter 1212 Load finish
chapter 1213 Load finish
chapter 1214 Loa

chapter 125 Load finish
chapter 126 Load finish
chapter 128 Load finish
chapter 129 Load finish
chapter 130 Load finish
chapter 131 Load finish
chapter 132 Load finish
chapter 133 Load finish
chapter 134 Load finish
chapter 135 Load finish
webtoon_title : 스위트홈
webtoon_id :  701081
weekday :  fri
webtoon_url : http://comic.naver.com/webtoon/list.nhn?titleId=701081&weekday=fri
chapter 14 Load finish
chapter 15 Load finish
chapter 16 Load finish
chapter 17 Load finish
chapter 18 Load finish
chapter 19 Load finish
chapter 20 Load finish
chapter 21 Load finish
chapter 22 Load finish
chapter 23 Load finish
chapter 24 Load finish
chapter 25 Load finish
chapter 26 Load finish
chapter 27 Load finish
chapter 28 Load finish
chapter 29 Load finish
webtoon_title : 더 게이머
webtoon_id :  552960
weekday :  fri
webtoon_url : http://comic.naver.com/webtoon/list.nhn?titleId=552960&weekday=fri
chapter 206 Load finish
chapter 207 Load finish
chapter 212 Load finish
chapter 214 Load finish
chapter 215 Load fi

chapter 72 Load finish
chapter 73 Load finish
chapter 74 Load finish
chapter 77 Load finish
chapter 78 Load finish
chapter 79 Load finish
chapter 80 Load finish
chapter 81 Load finish
webtoon_title : 몽홀
webtoon_id :  644112
weekday :  fri
webtoon_url : http://comic.naver.com/webtoon/list.nhn?titleId=644112&weekday=fri
chapter 147 Load finish
chapter 148 Load finish
chapter 149 Load finish
chapter 150 Load finish
chapter 152 Load finish
chapter 155 Load finish
chapter 156 Load finish
chapter 157 Load finish
chapter 158 Load finish
chapter 159 Load finish
chapter 160 Load finish
chapter 161 Load finish
webtoon_title : 강변살다
webtoon_id :  578108
weekday :  fri
webtoon_url : http://comic.naver.com/webtoon/list.nhn?titleId=578108&weekday=fri
chapter 141 Load finish
chapter 142 Load finish
chapter 143 Load finish
chapter 144 Load finish
chapter 145 Load finish
chapter 146 Load finish
chapter 147 Load finish
chapter 148 Load finish
chapter 149 Load finish
chapter 150 Load finish
chapter 151 Lo

chapter 9 Load finish
chapter 10 Load finish
chapter 11 Load finish
webtoon_title : 어글리후드
webtoon_id :  703630
weekday :  sat
webtoon_url : http://comic.naver.com/webtoon/list.nhn?titleId=703630&weekday=sat
chapter 5 Load finish
chapter 6 Load finish
chapter 7 Load finish
chapter 8 Load finish
chapter 10 Load finish
chapter 11 Load finish
chapter 12 Load finish
chapter 14 Load finish
chapter 15 Load finish
chapter 16 Load finish
chapter 17 Load finish
chapter 18 Load finish
chapter 19 Load finish
chapter 20 Load finish
webtoon_title : 놓지마 정신줄 시즌2
webtoon_id :  682637
weekday :  sat
webtoon_url : http://comic.naver.com/webtoon/list.nhn?titleId=682637&weekday=sat
chapter 166 Load finish
chapter 167 Load finish
chapter 168 Load finish
chapter 171 Load finish
chapter 172 Load finish
chapter 173 Load finish
chapter 174 Load finish
chapter 175 Load finish
chapter 177 Load finish
chapter 180 Load finish
chapter 181 Load finish
webtoon_title : 아메리카노 엑소더스
webtoon_id :  622644
weekday :  sat
web

chapter 135 Load finish
chapter 136 Load finish
chapter 137 Load finish
chapter 138 Load finish
chapter 139 Load finish
chapter 141 Load finish
chapter 142 Load finish
chapter 143 Load finish
chapter 144 Load finish
chapter 145 Load finish
chapter 146 Load finish
chapter 148 Load finish
chapter 150 Load finish
webtoon_title : 타인은 지옥이다
webtoon_id :  708378
weekday :  sun
webtoon_url : http://comic.naver.com/webtoon/list.nhn?titleId=708378&weekday=sun
chapter -1 Load finish
chapter 0 Load finish
chapter 1 Load finish
chapter 2 Load finish
chapter 4 Load finish
chapter 5 Load finish
chapter 6 Load finish
chapter 7 Load finish
chapter 8 Load finish
chapter 9 Load finish
chapter 11 Load finish
chapter 12 Load finish
chapter 13 Load finish
chapter 14 Load finish
webtoon_title : 열렙전사
webtoon_id :  670152
weekday :  sun
webtoon_url : http://comic.naver.com/webtoon/list.nhn?titleId=670152&weekday=sun
chapter 103 Load finish
chapter 104 Load finish
chapter 105 Load finish
chapter 106 Load finish

chapter 38 Load finish
chapter 39 Load finish
chapter 40 Load finish
chapter 41 Load finish
chapter 42 Load finish
chapter 43 Load finish
chapter 44 Load finish
chapter 45 Load finish
chapter 46 Load finish
chapter 47 Load finish
chapter 48 Load finish
chapter 49 Load finish
chapter 50 Load finish
chapter 51 Load finish
chapter 52 Load finish
chapter 53 Load finish
webtoon_title : 허니허니 웨딩
webtoon_id :  697537
weekday :  sun
webtoon_url : http://comic.naver.com/webtoon/list.nhn?titleId=697537&weekday=sun
chapter 31 Load finish
chapter 32 Load finish
chapter 33 Load finish
chapter 34 Load finish
chapter 35 Load finish
chapter 36 Load finish
chapter 37 Load finish
chapter 38 Load finish
chapter 39 Load finish
chapter 40 Load finish
chapter 41 Load finish
chapter 42 Load finish
chapter 43 Load finish
chapter 44 Load finish
chapter 45 Load finish
webtoon_title : 내일
webtoon_id :  695796
weekday :  sun
webtoon_url : http://comic.naver.com/webtoon/list.nhn?titleId=695796&weekday=sun
chapter 34